In [ ]:
from serial import Serial
import csv
import time
import datetime
import matplotlib.pyplot as plt

In [ ]:
arduino_port = "COM3"
arduino_baudrate = 115200
arduino_serial = Serial(arduino_port, arduino_baudrate)
data_header = ["LL", "LR", "DT", "AX", "AY", "AZ"]
file_path = ""

LL_values = []
LR_values = []

plt.ion()
fig, ax = plt.subplots()

try:
    with open(file_path + datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S") + ".csv",mode="w",newline="") as file:
        csv_writer = csv.writer(file)
        csv_writer.writerow(data_header)
        start_time = time.time()

        while time.time() - start_time < 120:
            data = arduino_serial.readline().decode("utf-8").strip()
            if data.count(",") == len(data_header) - 1:
                try:
                    LL, LR, DT, AX, AY, AZ = map(lambda x: float(x[3:]),data.split(","),)
                    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                    csv_writer.writerow([timestamp, LL, LR, DT, AX, AY, AZ])
                    LL_values.append(LL)
                    LR_values.append(LR)
                    ax.clear()
                    ax.plot(LL_values, label='LL')
                    ax.plot(LR_values, label='LR')
                    ax.legend(loc='')
                    plt.draw()
                except ValueError:
                    print(f"Invalid data format: {data}")
            else:
                print(f"Invalid data format: {data}")
            time.sleep(1)
except KeyboardInterrupt:
    arduino_serial.close()